In [8]:
import time
from numbers import Number
from functools import partial
from database.select.db_select import get_entire_season
from data.data import Game, Team, TeamGame

In [9]:
season_info, teams, games = await get_entire_season(2023)

In [10]:
def game_analysis(teams: list[Team], games: list[Game], regular_season_games: int) -> tuple[dict[str, Team], dict[str, list[TeamGame]]]:
    # map team names to team objects
    team_dict: dict[str, Team] = {team.full_name: team for team in teams}
    # map team names to their games played
    team_game_dict: dict[str, list[TeamGame]] = {}
    
    # calculate team stats based on all games
    for game in games:
        # initial values from game
        away_team: str = game.away_team
        home_team: str = game.home_team
        division_game: bool = team_dict[away_team].division == team_dict[home_team].division
        conference_game: bool = team_dict[away_team].conference == team_dict[home_team].conference
        playoff_game: bool = game.week > regular_season_games
        
        # create team game lists if they don't exist
        if away_team not in team_game_dict:
            team_game_dict[away_team] = []
        if home_team not in team_game_dict:
            team_game_dict[home_team] = []
        
        # points stats for away team
        team_dict[away_team].points_for += game.away_score
        team_dict[away_team].points_against += game.home_score
        team_dict[away_team].point_differential += game.away_score - game.home_score
        if conference_game:
            team_dict[away_team].points_for_in_conference_games += game.away_score
            team_dict[away_team].points_against_in_conference_games += game.home_score
            
        # points stats for home team
        team_dict[home_team].points_for += game.home_score
        team_dict[home_team].points_against += game.away_score
        team_dict[home_team].point_differential += game.home_score - game.away_score
        if conference_game:
            team_dict[home_team].points_for_in_conference_games += game.home_score
            team_dict[home_team].points_against_in_conference_games += game.away_score
            
        ##############################
        # win/loss/tie stats
        ##############################
        
        # handle win/loss stats if away team wins
        if game.away_score > game.home_score:
            away_result: chr = 'W'
            home_result: chr = 'L'
            team_dict[away_team].wins += 1
            team_dict[home_team].losses += 1
            if conference_game:
                team_dict[away_team].conference_wins += 1
                team_dict[home_team].conference_losses += 1
                if division_game:
                    team_dict[away_team].division_wins += 1
                    team_dict[home_team].division_losses += 1
        # handle win/loss stats if home team team wins
        elif game.away_score < game.home_score:
            away_result: chr = 'L'
            home_result: chr = 'W'
            team_dict[away_team].losses += 1
            team_dict[home_team].wins += 1
            if conference_game:
                team_dict[away_team].conference_losses += 1
                team_dict[home_team].conference_wins += 1
                if division_game:
                    team_dict[away_team].division_losses += 1
                    team_dict[home_team].division_wins += 1
        # handle tie stats if the game ends in a tie
        else:
            away_result: chr = 'T'
            home_result: chr = 'T'
            team_dict[away_team].ties += 1
            team_dict[home_team].ties += 1
            if conference_game:
                team_dict[away_team].conference_ties += 1
                team_dict[home_team].conference_ties += 1
                if division_game:
                    team_dict[away_team].division_ties += 1
                    team_dict[home_team].division_ties += 1
        
        # create away TeamGame and add to list of team games
        team_game_dict[away_team].append(
            TeamGame(
                game.week,
                game.week_name,
                home_team,
                game.away_score,
                game.home_score,
                away_result,
                False,
                division_game,
                conference_game,
                playoff_game)
        )
        
        # create home TeamGame and add to list of team games
        team_game_dict[home_team].append(
            TeamGame(
                game.week,
                game.week_name,
                away_team,
                game.home_score,
                game.away_score,
                home_result,
                True,
                division_game,
                conference_game,
                playoff_game
            )
        )
        
    return team_dict, team_game_dict

teams, team_games = game_analysis(teams, games, season_info.regular_season_week_count)

### Team Points Rankings

In [ ]:
def get_stat_value(team: Team, stat_type: str) -> int:
    if stat_type.startswith('points_for'):
        return team.points_for
    elif stat_type.startswith('points_against'):
        return team.points_against
    elif stat_type == 'combined_points_rank':
        return team.offensive_rank + team.defensive_rank
    # conference combined rank
    elif stat_type.startswith('combined_points_rank'):
        return team.offensive_rank_in_conference + team.defensive_rank_in_conference

def set_team_point_ranking(teams: dict[str, Team], team_name: str, stat_type: str, stat_ranking: int) -> dict[str, Team]:
    if stat_type == 'points_for':
        teams[team_name].offensive_rank = stat_ranking
    elif stat_type == 'points_against':
        teams[team_name].defensive_rank = stat_ranking
    # conference offensive rank
    elif stat_type.startswith('points_for'):
        teams[team_name].offensive_rank_in_conference = stat_ranking
    # conference defensive rank
    elif stat_type.startswith('points_against'):
        teams[team_name].defensive_rank_in_conference = stat_ranking
    elif stat_type == 'combined_points_rank':
        teams[team_name].combined_score_ranking = stat_ranking
    # conference combined rank
    elif stat_type.startswith('combined_points_rank'):
        teams[team_name].combined_score_ranking_in_conference = stat_ranking

def set_team_points_rankings(teams: dict[str, Team]) -> dict[str, Team]:
    stat_types: list[str] = [
        'points_for',
        'points_against',
        'points_for_AFC',
        'points_for_NFC',
        'points_against_AFC',
        'points_against_NFC',
        'combined_points_rank',
        'combined_points_rank_AFC',
        'combined_points_rank_NFC'
    ]
    
    # set all points stats
    for stat_type in stat_types:
        curr_rank: int = 1
        previous_points: int = -1
        conference = stat_type[-3:]
        is_conference_stat: bool = conference == 'AFC' or conference == 'NFC'
        
        # filter by conference if conference stat, otherwise all teams
        teams_to_rank: list[Team] = [team for team in teams.values() if team.conference == conference] if is_conference_stat else list(teams.values())
        # sort teams by stat
        sorted_teams: list[Team] = sorted(teams_to_rank, key=lambda team: (get_stat_value(team, stat_type), team.full_name), reverse=True)
        
        # set stat rankings for each team
        for i, team in enumerate(sorted_teams):
            stat_value = 0
            
            if stat_value != previous_points:
                previous_points = get_stat_value(team, stat_type)
                curr_rank += i
                
            teams = set_team_point_ranking(teams, team.full_name, stat_type, curr_rank)
            
    return teams

teams = set_team_points_rankings(teams)

### Win Percentage Stats

In [ ]:
def win_percentage(wins: int, losses: int, ties: int) -> float:
    num_games: int = wins + losses + ties
    
    if num_games == 0:
        return 0.0
    
    return round((wins + (ties * 0.5)) / num_games, 3)

divisions: dict[str, list[Team]] = {}
conferences: dict[str, list[Team]] = {}

def team_stats_from_games(team: Team, team_games: list[TeamGame], teams: dict[str, Team]) -> Team:
    # if there are no games for this team, values are 0
    if len(team_games) == 0:
        team.strength_of_victory = 0.0
        team.strength_of_schedule = 0.0
    
    # stored values from for loop
    sum_opponent_win_percentages: float = 0.0 # for strength of schedule
    sum_opponent_win_percentages_in_victory: float = 0.0 # for strength of victory
    
    # loop through team games and add to calculated stats where applicable
    for team_game in team_games:
        opponent_win_percentage: float = teams[team_game.opponent].win_percentage
        sum_opponent_win_percentages += opponent_win_percentage
        
        # strength of victory only applies to games won
        if team_game.result == 'W':
            sum_opponent_win_percentages_in_victory += opponent_win_percentage
    
    # add values to team object
    team.strength_of_victory = sum_opponent_win_percentages_in_victory / teams[team.wins] if team.wins > 0 else 0.0
    team.strength_of_schedule = sum_opponent_win_percentages / len(team_games)
    
    return team

for team, team_info in teams.items():
    # win percentage for each type of game
    team_info.win_percentage = win_percentage(team_info.wins, team_info.losses, team_info.ties)
    team_info.division_win_percentage = win_percentage(team_info.division_wins, team_info.division_losses, team_info.division_ties)
    team_info.conference_win_percentage = win_percentage(team_info.conference_wins, team_info.conference_losses, team_info.conference_ties)
    
    # fill in stats that require looping through each game played
    team_info = team_stats_from_games(team_info, team_games[team], teams)
    
    if team_info.division not in divisions:
        divisions[team_info.division] = []
    if team_info.conference not in conferences:
        conferences[team_info.conference] = []
        
    divisions[team_info.division].append(team_info)
    conferences[team_info.conference].append(team_info)

### Tiebreaker Helper Functions

In [ ]:
def create_ranking(sorted_team_stats: list[tuple[Team, Number]], starting_rank: int) -> dict[int, list[Team]]:
    # Create a dictionary to store the teams and their rank
    team_rankings: dict[int, list[Team]] = {}
    curr_rank: int = starting_rank
    previous_stat_value: Number = sorted_team_stats[0][0]
    
    # loop through each team mapped to a stat and create mapping of stat rank to team(s)
    for i, team_stats in enumerate(sorted_team_stats):
        team = team_stats[0]
        stat = team_stats[1]
        
        # if a new stat value is found, set the new value and add one to the current rank
        if stat < previous_stat_value:
            previous_stat_value = stat
            curr_rank += i + starting_rank
        
        # if there is no team assigned to this rank yet, add it to the dictionary
        if curr_rank not in team_rankings:  
            team_rankings[curr_rank] = [team]  
        # if another team has already been assigned to this rank, add it to the list for this rank
        else:
            team_rankings[curr_rank].append(team)
            
        print(f'\t{curr_rank} {team.full_name} ({stat})')
            
    return team_rankings

def win_percentage_from_games(team_games: list[TeamGame]) -> float:
    game_counts = {
        'W': 0,
        'L': 0,
        'T': 0
    }
    
    for game in team_games:
        game_counts[game.result] += 1
    
    return win_percentage(**game_counts)

def filter_games(teams: set[str], team_games: list[TeamGame]) -> list[TeamGame]:
    return [game for game in team_games if game.opponent in teams]
    
def win_percentage_from_certain_games(teams: set[str], team_games: list[TeamGame], min_games: int = 1) -> float:
    # determine the head to head games for the team
    filtered_games = filter_games(teams, team_games)
    
    # if there are no head to head games for this team, skip it
    if len(filtered_games) < min_games:
        return None
    
    return win_percentage_from_games(filtered_games)

def determine_common_teams(all_team_games: list[list[TeamGame]]) -> set[str]:
    # create a set of teams that each team has played
    teams_played: list[set[str]] = [set([game.opponent for game in team_games]) for team_games in all_team_games]
    
    # determine the common teams between all the sets
    return set.intersection(*teams_played)

def filtered_games_rankings(teams: list[Team], filter_teams: set[str], team_games: dict[str, list[TeamGame]], starting_rank: int, min_common_games: int = 1) -> dict[int, list[Team]]:
    # head to head win percentages
    win_percentages: list[float] = [win_percentage_from_certain_games(filter_teams, set(team_games[team.full_name]), min_common_games) for team in teams]
    
    # if there are no head to head games for this team, skip this tie breaker and return original rankings
    if None in win_percentages:
        return {starting_rank: teams}
    
    # map the win percentages to the team names
    team_tiebreaker_pairings: list[tuple[int, Team]] = [(win_percentage, team) for win_percentage in win_percentages]
        
    # Return the dictionary of teams and their rank
    return create_ranking(team_tiebreaker_pairings, starting_rank)

### Tiebreakers

In [ ]:
def win_percentage_ranking(teams: list[Team], starting_rank: int = 1) -> dict[int, list[Team]]:
    # Sort the list of teams by win percentage
    teams_by_win_percentage: list[Team] = sorted(teams, key=lambda x: x.win_percentage, reverse=True)
    team_tiebreaker_pairings: list[tuple[int, Team]] = [(team.win_percentage, team) for team in teams_by_win_percentage] 

    # Return the dictionary of teams and their rank
    return create_ranking(team_tiebreaker_pairings, starting_rank)

def head_to_head_ranking(teams: list[Team], team_games: dict[str, list[TeamGame]], starting_rank: int) -> dict[int, list[Team]]:
    # get the list of team names from the teams list for head to head tiebreaker
    team_names: set[str] = set([team.full_name for team in teams])
    
    return filtered_games_rankings(teams, team_names, team_games, starting_rank)

def head_to_head_sweep_ranking(teams: list[Team], team_games: dict[str, list[TeamGame]], starting_rank: int) -> dict[int, list[Team]]:
    # get the list of team names from the teams list for head to head tiebreaker
    team_names: set[str] = set([team.full_name for team in teams])
    
    for team in teams:
        filtered_games: list[TeamGame] = filter_games(team_names, team_games[team.full_name])
        
        # if head to head sweep
        if all(game.result for game in filtered_games):
            # set starting rank to team that sweeped, and set the next rank to list of other teams
            return {
                starting_rank: [team],
                starting_rank + 1: teams.copy().remove(team)
            }
    
    # if no head to head sweep, skip this tie breaker and return original rankings
    return {starting_rank: teams}
    
def common_games_ranking(teams: list[Team], team_games: dict[str, list[TeamGame]], starting_rank: int, min_common_games: int) -> dict[int, list[Team]]:
    # get the teams that each team has played
    common_teams: set[str] = determine_common_teams(list(team_games.values()))
    
    return filtered_games_rankings(teams, common_teams, team_games, starting_rank, min_common_games)

def division_win_percentage_ranking(teams: list[Team], starting_rank: int) -> dict[int, list[Team]]:
    # Sort the list of teams by division win percentage
    teams_by_division_win_percentage: list[Team] = sorted(teams, key=lambda x: x.division_win_percentage, reverse=True)
    team_tiebreaker_pairings: list[tuple[int, Team]] = [(team.division_win_percentage, team) for team in teams_by_division_win_percentage]
    
    # Return the dictionary of teams and their rank
    return create_ranking(team_tiebreaker_pairings, starting_rank)

def conference_win_percentage_ranking(teams: list[Team], starting_rank: int) -> dict[int, list[Team]]:
    # Sort the list of teams by conference win percentage
    teams_by_conference_win_percentage: list[Team] = sorted(teams, key=lambda x: x.conference_win_percentage, reverse=True)
    team_tiebreaker_pairings: list[tuple[int, Team]] = [(team.conference_win_percentage, team) for team in teams_by_conference_win_percentage]
    
    # Return the dictionary of teams and their rank
    return create_ranking(team_tiebreaker_pairings, starting_rank)

def strength_of_victory_ranking(teams: list[Team], starting_rank: int) -> dict[int, list[Team]]:
    # Sort the list of teams by strength of victory
    teams_by_strength_of_victory: list[Team] = sorted(teams, key=lambda x: x.strength_of_victory, reverse=True)
    team_tiebreaker_pairings: list[tuple[int, Team]] = [(team.strength_of_victory, team) for team in teams_by_strength_of_victory]
    
    # Return the dictionary of teams and their rank
    return create_ranking(team_tiebreaker_pairings, starting_rank)

def strength_of_schedule_ranking(teams: list[Team], starting_rank: int) -> dict[int, list[Team]]:
    # Sort the list of teams by strength of schedule
    teams_by_strength_of_schedule: list[Team] = sorted(teams, key=lambda x: x.strength_of_schedule, reverse=True)
    team_tiebreaker_pairings: list[tuple[int, Team]] = [(team.strength_of_schedule, team) for team in teams_by_strength_of_schedule]
    
    # Return the dictionary of teams and their rank
    return create_ranking(team_tiebreaker_pairings, starting_rank)

def combined_ranking_among_conference_ranking(teams: list[Team], starting_rank: int) -> dict[int, list[Team]]:
    # Sort the list of teams by "combined ranking among conference teams"
    teams_by_combined_ranking_among_conference: list[Team] = sorted(teams, key=lambda x: x.combined_score_ranking_in_conference, reverse=True)
    team_tiebreaker_pairings: list[tuple[int, Team]] = [(team.combined_score_ranking_in_conference, team) for team in teams_by_combined_ranking_among_conference]
    
    # Return the dictionary of teams and their rank
    return create_ranking(team_tiebreaker_pairings, starting_rank)

def combined_ranking_among_all_teams_ranking(teams: list[Team], starting_rank: int) -> dict[int, list[Team]]:
    # Sort the list of teams by "combined ranking among all teams"\
    teams_by_combined_ranking_among_all_teams: list[Team] = sorted(teams, key=lambda x: x.combined_score_ranking, reverse=True)
    team_tiebreaker_pairings: list[tuple[int, Team]] = [(team.combined_score_ranking, team) for team in teams_by_combined_ranking_among_all_teams]
    
    # Return the dictionary of teams and their rank
    return create_ranking(team_tiebreaker_pairings, starting_rank)

def net_points_in_conference_ranking(teams: list[Team], starting_rank: int) -> dict[int, list[Team]]:
    # Sort the list of teams by net points in conference
    teams_by_net_points_in_conference: list[Team] = sorted(teams, key=lambda x: x.points_for_in_conference_games, reverse=True)
    team_tiebreaker_pairings: list[tuple[int, Team]] = [(team.points_for_in_conference_games, team) for team in teams_by_net_points_in_conference]
    
    # Return the dictionary of teams and their rank
    return create_ranking(team_tiebreaker_pairings, starting_rank)

def net_points_all_games_ranking(teams: list[Team], starting_rank: int) -> dict[int, list[Team]]:
    # Sort the list of teams by net points in all games
    teams_by_net_points_all_games: list[Team] = sorted(teams, key=lambda x: x.points_for, reverse=True)
    team_tiebreaker_pairings: list[tuple[int, Team]] = [(team.points_for, team) for team in teams_by_net_points_all_games]
    
    # Return the dictionary of teams and their rank
    return create_ranking(team_tiebreaker_pairings, starting_rank)

def top_team_in_division_ranking(teams: list[Team], starting_rank: int) -> dict[int, list[Team]]:
    highest_ranking_team_in_division: dict[str, Team] = {}
    multiple_teams_in_same_division: bool = False
    
    for team in teams:
        if team.division not in highest_ranking_team_in_division:
            highest_ranking_team_in_division[team.division] = team
        else:
            multiple_teams_in_same_division = True
            if team.division_rank > highest_ranking_team_in_division[team.division].division_rank:
                highest_ranking_team_in_division[team.division] = team
    
    # if there are not multiple teams the same division in this tiebreaker, return original rankings
    if not multiple_teams_in_same_division:
        return {starting_rank: teams}
    
    # highest ranking teams in division
    highest_ranking_teams: list[Team] = [team for team in teams if highest_ranking_team_in_division[team.division] == team]
    # teams that are not the highest ranking team in division
    other_teams: list[Team] = [team for team in teams if team not in highest_ranking_teams]
    
    # seperate out the teams that are the highest ranking team in their division from the ones that aren't
    return {
        starting_rank: highest_ranking_teams,
        starting_rank + 1: other_teams
    }
    
def coin_toss_ranking(teams: list[Team], starting_rank: int) -> dict[int, list[Team]]:
    # Sort the list of teams by name in place of coin toss
    return {starting_rank + i: [team] for i, team in enumerate(sorted(teams, key=lambda x: x.full_name, reverse=True))}

In [ ]:
def create_partial_functions(team_games: dict[str, list[TeamGame]]) -> dict[str, partial[dict[int, list[Team]]]]:
    return {
        'head_to_head': partial(head_to_head_ranking, team_games=team_games),
        'head_to_head_sweep': partial(head_to_head_sweep_ranking, team_games=team_games),
        'division_win_percentage': partial(division_win_percentage_ranking),
        'common_games': partial(common_games_ranking, team_games=team_games),
        'conference_win_percentage': partial(conference_win_percentage_ranking),
        'strength_of_victory': partial(strength_of_victory_ranking),
        'strength_of_schedule': partial(strength_of_schedule_ranking),
        'combined_ranking_among_conference': partial(combined_ranking_among_conference_ranking),
        'combined_ranking_among_all_teams': partial(combined_ranking_among_all_teams_ranking),
        'net_points_in_conference': partial(net_points_in_conference_ranking),
        'net_points_all_games': partial(net_points_all_games_ranking),
        'top_team_in_division': partial(top_team_in_division_ranking),
        # 'new_touchdowns_in_all_games', partial(new_touchdowns_in_all_games_ranking), # not used yet
        'coin_toss': partial(coin_toss_ranking)
    }
    
division_tiebreakers: list[str] = [
    'head_to_head',
    'division_win_percentage',
    'common_games',
    'conference_win_percentage',
    'strength_of_victory',
    'strength_of_schedule',
    'compined_ranking_among_conference',
    'combined_ranking_among_all_teams',
    'net_points_in_conference',
    'net_points_all_games',
    # 'new_touchdowns_in_all_games', partial(new_touchdowns_in_all_games_ranking), # not used yet
    'coin_toss'
]

conference_tiebreakers: list[str] = [
    'top_team_in_division'
    'head_to_head_sweep',
    'conference_win_percentage',
    'common_games',
    'strength_of_victory',
    'strength_of_schedule',
    'compined_ranking_among_conference',
    'combined_ranking_among_all_teams',
    'net_points_in_conference',
    'net_points_all_games',
    # 'new_touchdowns_in_all_games', partial(new_touchdowns_in_all_games_ranking), # not used yet
    'coin_toss'
]

def get_rankings(teams: list[Team], team_games: dict[str, list[TeamGame]], tiebreakers: list[str], starting_rank: int = 1) -> list[tuple[str, int]]:
    team_rankings: dict[int, list[Team]] = win_percentage_ranking(teams, starting_rank)
    
    num_ties = len(teams) - len(team_rankings)
    
    func_mappings: dict[str, partial[dict[int, list[Team]]]] = create_partial_functions(teams, team_games)
    
    while num_ties > 0:
        try:
            # get the ranking where there is a tie
            tie_ranking = next(rank for rank, teams_at_rank in team_rankings.items() if len(teams_at_rank) > 1)
        except StopIteration:
            print('No ties found unexpectedly')
            break
        
        # run tiebreakers
        for tiebreaker in tiebreakers:
            new_team_rankings: dict[int, list[Team]] = func_mappings[tiebreaker](team=team, starting_rank=tie_ranking)

            # if a tie was broken, update rankings
            if len(teams) - len(new_team_rankings) < num_ties:
                num_ties = len(teams) - len(new_team_rankings)
                team_rankings.update(new_team_rankings)
                break
    
    # team-rank mappings
    return [(team[0].full_name, rank) for rank, team in team_rankings.items()]

### Run Tiebreakers

In [17]:
# division rankings
for division, team_list in divisions.items():
    print(division)
    team_rankings: list[tuple[str, int]] = get_rankings(team_list, team_games, division_tiebreakers)
    
    for team_name, rank in team_rankings:
        teams[team_name].division_rank = rank
        print(f'\t{team_name}: {rank}')

print()
  
# conference playoff rankings
for conference, team_list in conferences.items():
    division_winners: list[Team] = [team for team in team_list if team.division_rank == 1]
    non_division_winners: list[Team] = [team for team in team_list if team.division_rank > 1]
    
    print(conference)
    division_winner_rankings: list[tuple[Team, int]] = get_rankings(division_winners, team_games, conference_tiebreakers)
    wild_card_rankings: list[tuple[Team, int]] = get_rankings(division_winners, team_games, conference_tiebreakers, 5)
    
    conference_rankings: list[tuple[Team, int]] = division_winner_rankings.extend(wild_card_rankings)
    
    for team_name, rank in conference_rankings:
        teams[team_name].playoff_rank = rank
        print(f'\t{team_name}: {rank}')
        
        if rank == 4:
            print('_'*85)

AFC East
	1 Buffalo Bills (0.632)
	2 Miami Dolphins (0.611)
	3 New York Jets (0.412)
	4 New England Patriots (0.235)
Has 0 ties
AFC West
	1 Kansas City Chiefs (0.714)
	2 Las Vegas Raiders (0.471)
	2 Denver Broncos (0.471)
	3 Los Angeles Chargers (0.294)
Has 1 ties
AFC North
	1 Baltimore Ravens (0.737)
	2 Cleveland Browns (0.611)
	3 Pittsburgh Steelers (0.556)
	4 Cincinnati Bengals (0.529)
Has 0 ties
AFC South
	1 Houston Texans (0.579)
	2 Jacksonville Jaguars (0.529)
	2 Indianapolis Colts (0.529)
	3 Tennessee Titans (0.353)
Has 1 ties
NFC East
	1 Dallas Cowboys (0.667)
	2 Philadelphia Eagles (0.611)
	3 New York Giants (0.353)
	4 Washington Commanders (0.235)
Has 0 ties
NFC West
	1 San Francisco 49ers (0.7)
	2 Los Angeles Rams (0.556)
	3 Seattle Seahawks (0.529)
	4 Arizona Cardinals (0.235)
Has 0 ties
NFC North
	1 Detroit Lions (0.7)
	2 Green Bay Packers (0.526)
	3 Minnesota Vikings (0.412)
	3 Chicago Bears (0.412)
Has 1 ties
NFC South
	1 New Orleans Saints (0.529)
	2 Tampa Bay Buccaneer